In [2]:
import sys
sys.path.append('..')
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer, seed_everything
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split

from torch_explain.models.explainer import MuExplainer
from experiments.data.load_datasets import load_vDem

ERROR! Session/line number was not unique in database. History logging moved to new session 1263


In [3]:
x, y, concept_names = load_vDem()

dataset = TensorDataset(x, y)

train_size = int(len(dataset) * 0.5)
val_size = (len(dataset) - train_size) // 2
test_size = len(dataset) - train_size - val_size
train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])
train_loader = DataLoader(train_data, batch_size=train_size)
val_loader = DataLoader(val_data, batch_size=val_size)
test_loader = DataLoader(test_data, batch_size=test_size)

n_concepts = next(iter(train_loader))[0].shape[1]
n_classes = 2

print(concept_names)
print(n_concepts)
print(n_classes)

Main 3 - Area 14 - Raw 62
Index(['v2x_api', 'v2x_mpi', 'v2x_freexp_altinf', 'v2x_frassoc_thick',
       'v2x_suffr', 'v2xel_frefair', 'v2x_elecoff', 'v2xcl_rol', 'v2x_cspart',
       'v2xdl_delib', 'v2x_egal', 'v2xeg_eqprotec', 'v2xeg_eqaccess',
       'v2xeg_eqdr'],
      dtype='object')
14
2


In [7]:
seed_everything(42)

base_dir = f'./results/vdem/explainer'
os.makedirs(base_dir, exist_ok=True)

n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
results_list = []
for split, (trainval_index, test_index) in enumerate(skf.split(x.cpu().detach().numpy(), y.argmax(dim=1).cpu().detach().numpy())):
    print(f'Split [{split + 1}/{n_splits}]')
    x_trainval, x_test = torch.FloatTensor(x[trainval_index]), torch.FloatTensor(x[test_index])
    y_trainval, y_test = torch.FloatTensor(y[trainval_index]), torch.FloatTensor(y[test_index])
    x_train, x_val, y_train, y_val = train_test_split(x_trainval, y_trainval, test_size=0.2, random_state=42)
    print(f'{len(y_train)}/{len(y_val)}/{len(y_test)}')

    train_data = TensorDataset(x_train, y_train)
    val_data = TensorDataset(x_val, y_val)
    test_data = TensorDataset(x_test, y_test)
    train_loader = DataLoader(train_data, batch_size=train_size)
    val_loader = DataLoader(val_data, batch_size=val_size)
    test_loader = DataLoader(test_data, batch_size=test_size)

    checkpoint_callback = ModelCheckpoint(dirpath=base_dir, monitor='val_loss', save_top_k=1)
    trainer = Trainer(min_epochs=200, max_epochs=200, gpus=1, auto_lr_find=True, deterministic=True,
                      check_val_every_n_epoch=1, default_root_dir=base_dir,
                      weights_save_path=base_dir, callbacks=[checkpoint_callback])
    model = MuExplainer(n_concepts=n_concepts, n_classes=n_classes, 
                        l1=0.001, lr=0.01, explainer_hidden=[50])

    trainer.fit(model, train_loader, val_loader)
    model.freeze()
    model_results = trainer.test(model, test_dataloaders=test_loader)
    results, results_full = model.explain_class(val_loader, test_loader, target_class=1,
                                                topk_explanations=1, max_minterm_complexity=20,
                                                concept_names=concept_names)
    results['model_accuracy'] = model_results[0]['test_acc']
    print(f"Explanation: {results_full[0]['explanation']}")
    print(f"Explanation: {results['explanation_accuracy']}")

    results_list.append(results)

Global seed set to 42
c:\users\pietr\anaconda3\envs\deep-logic\lib\site-packages\pytorch_lightning\utilities\distributed.py:52: UserWarning: Checkpoint directory ./results/vdem/explainer exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | loss  | NLLLoss    | 0     
1 | model | Sequential | 1.6 K 
-------------------------------------
1.6 K     Trainable params
0         Non-trainable params
1.6 K     Total params
0.006     Total estimated model params size (MB)


Split [1/10]
2436/610/339


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.9349112510681152, 'test_acc_epoch': 0.9349112510681152}
--------------------------------------------------------------------------------
Explanation: v2xel_frefair
Explanation: 0.8994082840236687


In [8]:
results_df = pd.DataFrame(results_list)
print(results_df)
# print(list(results_df['explanation'].values))
print(results_df.mean())
results

   explanation_accuracy  explanation_fidelity  explanation_complexity  \
0              0.495575              0.949853                     3.0   
1              0.902655              0.418879                     1.0   
2              0.920354              0.466077                     1.0   
3              0.920354              0.442478                     2.0   
4              0.920354              0.418879                     1.0   
5              0.896450              0.428994                     1.0   
6              0.896450              0.428994                     1.0   
7              0.923077              0.437870                     2.0   
8              0.896450              0.434911                     1.0   
9              0.899408              0.458580                     1.0   

   model_accuracy  
0        0.949853  
1        0.938053  
2        0.938053  
3        0.949853  
4        0.958702  
5        0.931953  
6        0.943787  
7        0.961538  
8        0.93491

{'explanation_accuracy': 0.8994082840236687,
 'explanation_fidelity': 0.45857988165680474,
 'explanation_complexity': 1.0,
 'model_accuracy': 0.9349112510681152}

In [9]:
dt_scores, rf_scores = [], []
for split, (trainval_index, test_index) in enumerate(skf.split(x.cpu().detach().numpy(), y.argmax(dim=1).cpu().detach().numpy())):
    print(f'Split [{split + 1}/{n_splits}]')
    x_trainval, x_test = x[trainval_index], x[test_index]
    y_trainval, y_test = y[trainval_index].argmax(dim=1), y[test_index].argmax(dim=1)

    dt_model = DecisionTreeClassifier(max_depth=3, random_state=split)
    dt_model.fit(x_trainval, y_trainval)
    dt_scores.append(dt_model.score(x_test, y_test))

    rf_model = RandomForestClassifier(random_state=split)
    rf_model.fit(x_trainval, y_trainval)
    rf_scores.append(rf_model.score(x_test, y_test))

print(f'Random forest scores: {np.mean(rf_scores)} (+/- {np.std(rf_scores)})')
print(f'Decision tree scores: {np.mean(dt_scores)} (+/- {np.std(dt_scores)})')
print(f'Mu net scores (model): {results_df["model_accuracy"].mean()} (+/- {results_df["model_accuracy"].std()})')
print(f'Mu net scores (exp): {results_df["explanation_accuracy"].mean()} (+/- {results_df["explanation_accuracy"].std()})')

Split [1/10]
Split [2/10]
Split [3/10]
Split [4/10]
Split [5/10]
Split [6/10]
Split [7/10]
Split [8/10]
Split [9/10]
Split [10/10]
Random forest scores: 0.9468188720741478 (+/- 0.009742803957421088)
Decision tree scores: 0.9364778062872006 (+/- 0.00894015987414031)
Mu net scores (model): 0.9441613912582397 (+/- 0.01039829211719923)
Mu net scores (exp): 0.8671126354924856 (+/- 0.1310488563045732)
